# Task 3 — High-Level to Low-Level Architecture Converter (Colab)

This notebook converts high-level business requirements into:
- Module decomposition (modules + responsibilities)
- Database schema (tables, fields, relations)
- Pseudocode for main flows

Flow:
1. Provide one or more high-level requirements (editable cell).
2. Optionally use OpenAI (set OPENAI_API_KEY in .env) to generate structured JSON.
3. If OpenAI is not available, a deterministic fallback generator will produce a useful output.
4. Save outputs as `task3_outputs/<req_name>.json` and `task3_outputs/<req_name>_report.md`.

Run cells in order. Each cell is self-contained and explained.

In [1]:
!pip install --quiet openai python-dotenv

In [2]:
%%writefile .env
OPENAI_API_KEY=

# Load env
from dotenv import load_dotenv
import os
load_dotenv('.env')
print("OPENAI_API_KEY present:", bool(os.getenv('OPENAI_API_KEY')))

Writing .env


In [3]:
import os, json, re, textwrap
from pathlib import Path
from dotenv import load_dotenv
load_dotenv('.env')

OUT = Path('task3_outputs')
OUT.mkdir(exist_ok=True)
print("Output folder:", OUT.resolve())

Output folder: /content/task3_outputs


In [4]:
requirements = [
    """Build an e-commerce product listing and checkout system where users can browse products, add to cart, and checkout using credit/debit or wallet. Admins must be able to add products, update inventory, and view orders. The system should support order history and basic analytics for top-selling products.""",

    """Create an internal expense reporting tool for employees: submit expense claims with receipts, manager approval flow, reimbursements by finance, and monthly summary reports for accounting.""",

    """Design a content ingestion pipeline that receives RSS and social media posts, extracts metadata, classifies by topic, stores raw and processed content, and serves topic-based feeds to a frontend."""
]

for i, r in enumerate(requirements):
    print(f"[{i}] {r[:200].replace('\\n',' ')}{'...' if len(r)>200 else ''}\n")

[0] Build an e-commerce product listing and checkout system where users can browse products, add to cart, and checkout using credit/debit or wallet. Admins must be able to add products, update inventory, ...

[1] Create an internal expense reporting tool for employees: submit expense claims with receipts, manager approval flow, reimbursements by finance, and monthly summary reports for accounting.

[2] Design a content ingestion pipeline that receives RSS and social media posts, extracts metadata, classifies by topic, stores raw and processed content, and serves topic-based feeds to a frontend.



In [9]:
import os
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
use_openai = bool(OPENAI_API_KEY)

def build_prompt(requirement_text):
    prompt = f"""
You are a senior software architect. Convert the following high-level business requirement into a JSON object with keys:
- modules: array of objects {{name, description}}
- schema: array of table objects {{{{name, columns: [{{{{name,type,pk,nullable,description}}}}]}}, relations: [{{{{from_table, from_field, to_table, to_field, type}}}}]}}}}
- pseudocode: object with keys being main flows and values being an array of step-by-step pseudocode lines
- notes: any assumptions or non-functional considerations

Return ONLY valid JSON (no extra commentary).

Requirement:
{requirement_text}
"""
    return prompt

def fallback_convert(requirement_text):
    text = requirement_text.lower()
    modules = []
    schema = []
    pseudocode = {}

    if any(k in text for k in ['e-commerce','checkout','cart','product','inventory','order']):
        modules = [
            {"name":"Auth","description":"User registration, login, and session management."},
            {"name":"Catalog","description":"Product listing, search, filters, and product details."},
            {"name":"Cart","description":"Temporary cart storage and manipulation."},
            {"name":"Checkout","description":"Order creation, payment integration, and order confirmation."},
            {"name":"Admin","description":"Product CRUD, inventory updates, and order management."},
            {"name":"Analytics","description":"Aggregate sales, top-selling products and reporting."}
        ]
        schema = [
            {"name":"users", "columns":[
                {"name":"id","type":"uuid","pk":True,"nullable":False,"description":"Primary key"},
                {"name":"email","type":"varchar(255)","pk":False,"nullable":False,"description":"User email"},
                {"name":"password_hash","type":"varchar(255)","pk":False,"nullable":False,"description":"Password hash"},
                {"name":"created_at","type":"timestamp","pk":False,"nullable":False,"description":"Account created time"}
            ], "relations":[]},
            {"name":"products","columns":[
                {"name":"id","type":"uuid","pk":True,"nullable":False,"description":"Product id"},
                {"name":"title","type":"varchar(255)","pk":False,"nullable":False,"description":"Product title"},
                {"name":"description","type":"text","pk":False,"nullable":True,"description":"Product description"},
                {"name":"price","type":"decimal(10,2)","pk":False,"nullable":False,"description":"Price"},
                {"name":"inventory","type":"integer","pk":False,"nullable":False,"description":"Stock count"}
            ], "relations":[]},
            {"name":"orders","columns":[
                {"name":"id","type":"uuid","pk":True,"nullable":False,"description":"Order id"},
                {"name":"user_id","type":"uuid","pk":False,"nullable":False,"description":"FK to users"},
                {"name":"total_amount","type":"decimal(10,2)","pk":False,"nullable":False,"description":"Total"},
                {"name":"status","type":"varchar(50)","pk":False,"nullable":False,"description":"Order status"},
                {"name":"created_at","type":"timestamp","pk":False,"nullable":False,"description":"Order time"}
            ], "relations":[{"from_table":"orders","from_field":"user_id","to_table":"users","to_field":"id","type":"many-to-one"}]}
        ]
        pseudocode = {
            "add_to_cart": [
                "Validate user session or allow guest cart",
                "Fetch product by id",
                "Check inventory >= requested quantity",
                "Create or update cart item with quantity"
            ],
            "checkout": [
                "Validate cart is not empty",
                "Calculate totals and taxes",
                "Validate payment info and call payment gateway",
                "On success: create order record, decrement product inventory, send confirmation email"
            ]
        }
    elif any(k in text for k in ['expense','reimburse','expense report','receipt','approval']):
        modules = [
            {"name":"Auth","description":"User auth and roles (employee, manager, finance)"},
            {"name":"Expense","description":"Submit expenses with receipts, edit, and view status"},
            {"name":"Approval","description":"Manager review & approve/reject flow"},
            {"name":"Finance","description":"Reimbursement processing and accounting export"},
            {"name":"Reporting","description":"Monthly summaries for accounting"}
        ]
        schema = [
            {"name":"employees","columns":[
                {"name":"id","type":"uuid","pk":True,"nullable":False,"description":"Employee id"},
                {"name":"name","type":"varchar(255)","pk":False,"nullable":False,"description":"Employee name"}
            ], "relations":[]},
            {"name":"expenses","columns":[
                {"name":"id","type":"uuid","pk":True,"nullable":False,"description":"Expense id"},
                {"name":"employee_id","type":"uuid","pk":False,"nullable":False,"description":"FK to employees"},
                {"name":"amount","type":"decimal(10,2)","pk":False,"nullable":False,"description":"Amount claimed"},
                {"name":"receipt_url","type":"varchar(500)","pk":False,"nullable":True,"description":"Receipt file"},
                {"name":"status","type":"varchar(50)","pk":False,"nullable":False,"description":"pending/approved/rejected"}
            ], "relations":[{"from_table":"expenses","from_field":"employee_id","to_table":"employees","to_field":"id","type":"many-to-one"}]}
        ]
        pseudocode = {
            "submit_expense": [
                "Validate employee identity",
                "Validate amount and receipt format",
                "Store expense with status 'pending' and attach receipt",
                "Notify manager for approval"
            ],
            "approve_expense": [
                "Manager reviews expense and receipt",
                "If approve: change status to 'approved' and notify finance",
                "Finance processes reimbursement"
            ]
        }
    schema = [
    {"name":"sources","columns":[
        {"name":"id","type":"uuid","pk":True,"nullable":False,"description":"Source id"},
        {"name":"name","type":"varchar(255)","pk":False,"nullable":False,"description":"Source name"},
        {"name":"type","type":"varchar(50)","pk":False,"nullable":False,"description":"rss/twitter/api"}
    ], "relations":[]},

    {"name":"content","columns":[
        {"name":"id","type":"uuid","pk":True,"nullable":False,"description":"Content id"},
        {"name":"source_id","type":"uuid","pk":False,"nullable":False,"description":"FK to sources"},
        {"name":"raw_text","type":"text","pk":False,"nullable":True,"description":"Original text"},
        {"name":"topic","type":"varchar(100)","pk":False,"nullable":True,"description":"Assigned topic"},
        {"name":"ingested_at","type":"timestamp","pk":False,"nullable":False,"description":"Ingest time"}
    ], "relations":[
        {"from_table":"content","from_field":"source_id","to_table":"sources","to_field":"id","type":"many-to-one"}
    ]}
]


    notes = ["Generated by deterministic fallback; refine types and constraints for production.", "Consider indexing frequently queried columns and handling soft deletes."]
    return {"modules": modules, "schema": schema, "pseudocode": pseudocode, "notes": notes}


In [11]:
def generate_with_openai(requirement_text, max_tokens=1000, temperature=0.2):
    import openai, re, json
    openai.api_key = OPENAI_API_KEY
    prompt = build_prompt(requirement_text)
    resp = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[{"role":"user","content":prompt}],
        max_tokens=max_tokens,
        temperature=temperature
    )
    out = resp['choices'][0]['message']['content']
    try:
        return json.loads(out)
    except Exception:
        m = re.search(r'\{[\s\S]*\}\s*$', out) or re.search(r'\{[\s\S]*\}', out)
        if m:
            try:
                return json.loads(m.group(0))
            except Exception as e:
                try:
                    cleaned = m.group(0).replace("'", '"')
                    return json.loads(cleaned)
                except Exception:
                    return {"raw_output": out}
        else:
            return {"raw_output": out}

In [12]:
results = []
for idx, req in enumerate(requirements):
    name = f"req_{idx+1}"
    safe_name = re.sub(r'[^a-zA-Z0-9_\\-]', '_', name)
    print("Processing:", safe_name)
    if use_openai:
        try:
            parsed = generate_with_openai(req)
            if not isinstance(parsed, dict) or not any(k in parsed for k in ('modules','schema','pseudocode')):
                print("OpenAI response missing expected keys; using fallback and attaching raw output.")
                fallback = fallback_convert(req)
                parsed = {"fallback": fallback, "raw_openai_output": parsed}
        except Exception as e:
            print("OpenAI call failed:", e)
            parsed = {"fallback": fallback_convert(req), "error": str(e)}
    else:
        parsed = {"fallback": fallback_convert(req)}
    out_file = OUT / f"{safe_name}.json"
    with open(out_file, 'w', encoding='utf-8') as f:
        json.dump(parsed, f, indent=2)
    print("Saved:", out_file)
    results.append((safe_name, out_file, parsed))
print("\\nAll done. Generated files:", [r[1].name for r in results])

Processing: req_1
Saved: task3_outputs/req_1.json
Processing: req_2
Saved: task3_outputs/req_2.json
Processing: req_3
Saved: task3_outputs/req_3.json
\nAll done. Generated files: ['req_1.json', 'req_2.json', 'req_3.json']


In [13]:
def pretty_schema_to_markdown(schema):
    md = ""
    for t in schema:
        md += f"### Table: {t.get('name')}\n\n"
        for c in t.get('columns', []):
            md += f"- **{c.get('name')}** ({c.get('type')})"
            if c.get('pk'): md += " — PK"
            if c.get('nullable'): md += " — nullable"
            md += f": {c.get('description','')}\n"
        if t.get('relations'):
            md += "\nRelations:\n"
            for rel in t.get('relations', []):
                md += f"- {rel}\n"
        md += "\n"
    return md

for safe_name, out_file, parsed in results:
    md_lines = []
    md_lines.append(f"# Report for {safe_name}\n")
    md_lines.append("## Input requirement\n")
    idx = int(safe_name.split('_')[-1]) - 1
    md_lines.append(requirements[idx] + "\n")
    md_lines.append("## Modules\n")
    modules = parsed.get('modules') or parsed.get('fallback',{}).get('modules',[])
    for m in modules:
        md_lines.append(f"- **{m.get('name')}**: {m.get('description')}\n")
    md_lines.append("\n## Schema\n")
    schema = parsed.get('schema') or parsed.get('fallback',{}).get('schema',[])
    md_lines.append(pretty_schema_to_markdown(schema))
    md_lines.append("## Pseudocode\n")
    pseudocode = parsed.get('pseudocode') or parsed.get('fallback',{}).get('pseudocode',{})
    for k,v in pseudocode.items():
        md_lines.append(f"### {k}\n")
        for step in v:
            md_lines.append(f"- {step}\n")
    md_lines.append("\n## Notes\n")
    notes = parsed.get('notes') or parsed.get('fallback',{}).get('notes', [])
    for n in notes:
        md_lines.append(f"- {n}\n")

    report_path = OUT / f"{safe_name}_report.md"
    report_path.write_text("\n".join(md_lines), encoding='utf-8')
    print("Saved report:", report_path)

Saved report: task3_outputs/req_1_report.md
Saved report: task3_outputs/req_2_report.md
Saved report: task3_outputs/req_3_report.md


In [14]:
def generate_erd_ascii(schema):
    erd = ""
    for t in schema:
        erd += f"+------------------+\n| {t['name']}\n"
        for c in t.get('columns', []):
            pk = ' (PK)' if c.get('pk') else ''
            erd += f"|  - {c['name']}: {c['type']}{pk}\n"
        erd += "+------------------+\n\n"
    rels = []
    for t in schema:
        for r in t.get('relations', []):
            rels.append(f"{r.get('from_table')}.{r.get('from_field')} -> {r.get('to_table')}.{r.get('to_field')} ({r.get('type')})")
    return erd, rels

for safe_name, out_file, parsed in results:
    schema = parsed.get('schema') or parsed.get('fallback',{}).get('schema',[])
    erd, rels = generate_erd_ascii(schema)
    erd_path = OUT / f"{safe_name}_erd.txt"
    erd_text = "ERD ASCII\n\n" + erd + ("\nRelations:\n" + "\n".join(rels) if rels else "\nNo explicit relations found.\n")
    erd_path.write_text(erd_text, encoding='utf-8')
    print("Saved ERD:", erd_path)

Saved ERD: task3_outputs/req_1_erd.txt
Saved ERD: task3_outputs/req_2_erd.txt
Saved ERD: task3_outputs/req_3_erd.txt


In [15]:
from google.colab import files
to_dl = []
for p in sorted(OUT.glob('*')):
    to_dl.append(str(p))
    print("Prepared for download:", p.name)
for fp in to_dl:
    files.download(fp)

Prepared for download: req_1.json
Prepared for download: req_1_erd.txt
Prepared for download: req_1_report.md
Prepared for download: req_2.json
Prepared for download: req_2_erd.txt
Prepared for download: req_2_report.md
Prepared for download: req_3.json
Prepared for download: req_3_erd.txt
Prepared for download: req_3_report.md


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>